# Lab 09: Embeddings and Semantic Search\n\n**Course:** Generative AI for Banking Sector  \n**Institution:** Banco Nacional de Costa Rica (BNCR)  \n**Instructor:** Manuela Larrea  \n**Duration:** 3 hours\n\n---\n\n## Learning Objectives\n\nBy the end of this lab, you will be able to:\n\n1. Understanding embeddings\n2. Azure OpenAI embeddings API\n3. Vector similarity search\n4. Building a document search system\n5. Semantic search for banking FAQs\n\n---

In [ ]:
import os\nimport sys\nfrom openai import AzureOpenAI\nfrom dotenv import load_dotenv\nimport json\n\nsys.path.append('../../utils')\nfrom azure_openai_helper import AzureOpenAIClient\n\nload_dotenv()\nclient = AzureOpenAIClient()\n\nprint("✓ Setup complete")

## Introduction\n\nIn this lab, we will explore **Embeddings and Semantic Search** and its applications in the banking sector.\n\nThis is a critical component for building production-ready GenAI systems at BNCR.

## Part 1: Understanding embeddings

In [ ]:
# TODO: Implement Understanding embeddings\n# Code examples will be provided during the live session\npass

## 🎯 Practical Exercises\n\n### Exercise 1: Build FAQ search engine\n### Exercise 2: Product recommendation system\n### Exercise 3: Document similarity finder

In [ ]:
# TODO: Complete the exercises\n# Your code here:\n

## Summary\n\nIn this lab, you learned about Embeddings and Semantic Search and how to apply it in banking scenarios.\n\n### Key Takeaways:\n\n- Understanding embeddings\n- Azure OpenAI embeddings API\n- Vector similarity search\n- Building a document search system\n- Semantic search for banking FAQs\n\n### Best Practices:\n\n1. Always validate inputs and outputs\n2. Implement proper error handling\n3. Monitor performance and costs\n4. Follow security best practices\n5. Document your implementations\n\n---\n\n**Instructor:** Manuela Larrea | manuela.larrea@idataglobal.com